### Rep Scraping

Grab Rep from Zip

In [340]:
# Pull rep from zipcode

import mechanize

# Insantiate browser
br = mechanize.Browser()
br.set_handle_robots(False)

# Open lookup link
br.open("https://ziplook.house.gov/")

# Find form and input zipcode
br.select_form(nr=1)
br.form["ZIP"] = "22182"
br.submit()

<response_seek_wrapper at 0x2a5a045a850 whose wrapped object = <closeable_response at 0x2a5a0446050 whose fp = <_io.BufferedReader name=4128>>>

Grab Rep Name from HTML

In [341]:
# Scrape for rep name

import requests
import lxml.html

# Open URL
URL = br.geturl()  
page = requests.get(URL, headers={
    "User-Agent" : "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/51.0.2704.103 Safari/537.36"})

# Scrape HTML
html = lxml.html.fromstring(page.content)
name = html.xpath("/html/body/div[2]/div/div[2]/section/div/div[2]/div[2]/div/div/p/a[1]")
name[0].text

'Gerald E. Connolly '

Grab Rep Infos With BS

In [ ]:
# BS Table Scraping
from bs4 import BeautifulSoup

soup: BeautifulSoup = BeautifulSoup(page.content, "html.parser")
table = soup.find('div', attrs = {'class':'col-xs-16 col-md-4 pull-md-12 repdistrict'}) 
if (len(name) > 1): 
    table = soup.find('div', attrs = {'id':'PossibleReps'}) 
table

<div class="col-xs-16 col-md-4 pull-md-12 repdistrict">
<h1>Find Your Representative</h1>
<p>
<strong>Information provided:</strong><br/>
<em>Zip code: 22182
</em><br/>

		is located in the 11th Congressional district of Virginia.

		<p>The representative for this district is:</p>
<div id="PossibleReps">
<div id="RepInfo" name="RepeatReps">
<p class="rep color4"><img alt="" border="0" class="repPhoto" src="/zip/pictures/va11_connolly.jpg"/><br/>
<a href="https://connolly.house.gov/">Gerald E. Connolly </a><br/>
                	Democrat<br/>
<a href="https://connolly.house.gov/"><img alt="Go to website" class="repLink" height="13" src="/content/static/img/icon-web.gif" width="13"/></a>
<a href="https://connolly.house.gov/contact/zip-auth.htm"><img alt="Go to contact form" class="repLink" height="11" src="/content/static/img/icon-email.gif" width="17"/></a>
</p>
</div>
</div>
</p></div>

In [343]:


info = table.findAll('div')[0].text.split()
info[-4]

'Gerald'

In [344]:
info = table.find('p')
district = str.splitlines(info.text)[5]
party = str.splitlines(info.text)[12]


In [345]:
import regex as re
party = re.sub(r'[\t ]', "", party)
party

'Democrat'

In [355]:
dist = re.match("([^0-9]*)([0-9].*)", district).group(2)
dist



'11th Congressional district of Virginia.'